# K-Means Clustering: A Comprehensive Tutorial
## DA5400W - Foundations of Machine Learning
### Instructor: Dr. Arun B Ayyar
### IIT Madras

---

## Overview

This notebook provides a detailed explanation of:
1. **Cluster Analysis** fundamentals
2. **K-Means Algorithm** - theory, implementation, and visualization
3. **Elbow Method** - determining the optimal number of clusters
4. **Practical Applications** with real datasets

**Learning Objectives:**
- Understand unsupervised learning and clustering
- Master the K-means algorithm
- Learn to select optimal k using the elbow method
- Implement K-means from scratch and with scikit-learn
- Apply K-means to real-world problems

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs, load_iris, load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score
from scipy.spatial.distance import cdist

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

# Set random seed
np.random.seed(42)

print("✓ All libraries imported successfully!")
print("✓ Ready to start the K-means tutorial")

---
# Part 1: Introduction to Cluster Analysis

## 1.1 What is Clustering?

**Clustering** is an unsupervised learning technique that partitions data into groups (clusters) such that:
- **High intra-cluster similarity**: Data points within the same cluster are similar
- **Low inter-cluster similarity**: Data points in different clusters are dissimilar

## 1.2 Key Definitions

**Centroid** (μᵢ): The mean of all data points in a cluster
$$\mu_i = \frac{1}{N_i} \sum_{x_j \in C_i} x_j$$

**Radius** (rᵢ): Maximum distance from any data point to the centroid
$$r_i = \max_{x_j \in C_i} ||x_j - \mu_i||^2$$

**Diameter** (dᵢ): Maximum pairwise distance among data points in a cluster
$$d_i = \max_{x_p, x_q \in C_i} ||x_p - x_q||^2$$

## 1.3 Applications

- **Marketing**: Customer segmentation
- **Documents**: Topic detection, information retrieval
- **Image Processing**: Image segmentation
- **Manufacturing**: Shop floor management
- **Security**: Anomaly detection

In [ ]:
# Visualize labeled vs unlabeled data
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Generate synthetic data
X, y_true = make_blobs(n_samples=300, centers=3, n_features=2, 
                       cluster_std=0.8, random_state=42)

# Labeled data (supervised learning)
colors = ['red', 'blue', 'green']
for i in range(3):
    mask = y_true == i
    axes[0].scatter(X[mask, 0], X[mask, 1], c=colors[i], 
                    label=f'Class {i}', s=60, alpha=0.7, edgecolor='black')
axes[0].set_title('Labeled Training Data\n(Supervised Learning)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Feature 1', fontsize=12)
axes[0].set_ylabel('Feature 2', fontsize=12)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Unlabeled data (unsupervised learning)
axes[1].scatter(X[:, 0], X[:, 1], c='darkred', s=60, alpha=0.7, edgecolor='black')
axes[1].set_title('Unlabeled Training Data\n(Unsupervised Learning - Clustering)', 
                  fontsize=14, fontweight='bold')
axes[1].set_xlabel('Feature 1', fontsize=12)
axes[1].set_ylabel('Feature 2', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Clustering discovers hidden structure in unlabeled data!")

---
# Part 2: K-Means Algorithm

## 2.1 Why K-Means?

- **Most widely used** clustering algorithm
- **Simple** to understand and implement
- **Reasonably fast** for small to medium datasets
- Provides a quick estimate of the "lay of the land"
- Available in most data mining tools

## 2.2 The K-Means Algorithm

**Objective**: Find K cluster centers that minimize the sum of squared distances of each data point to its nearest cluster center.

**Objective Function (Inertia/Within-Cluster Sum of Squares)**:
$$J = \sum_{i=1}^{K} \sum_{x_j \in C_i} ||x_j - \mu_i||^2$$

**Algorithm Steps**:

**Input**: Number of clusters k (predefined)

1. **Initialize**: Choose k cluster centers (randomly or using heuristics)
2. **Assignment Step**: Assign each data point to the nearest cluster center
   $$C_i = \{x_j : ||x_j - \mu_i|| \leq ||x_j - \mu_l|| \text{ for all } l\}$$
3. **Update Step**: Recompute cluster centers as the mean of assigned points
   $$\mu_i = \frac{1}{|C_i|} \sum_{x_j \in C_i} x_j$$
4. **Repeat**: Go to step 2 until convergence (centroids don't change significantly)

**Convergence**: The algorithm converges when:
- Centroids stop moving, OR
- Cluster assignments don't change, OR
- Maximum iterations reached

## 2.3 K-Means Visualization: Step-by-Step

In [ ]:
# Generate simple 2D data for visualization
X_demo, _ = make_blobs(n_samples=150, centers=3, n_features=2, 
                       cluster_std=0.6, random_state=42)

# Initialize 3 random centroids
k = 3
np.random.seed(10)
initial_centroids = X_demo[np.random.choice(X_demo.shape[0], k, replace=False)]

def assign_clusters(X, centroids):
    """Assign each point to nearest centroid"""
    distances = cdist(X, centroids, 'euclidean')
    return np.argmin(distances, axis=1)

def update_centroids(X, labels, k):
    """Compute new centroids as mean of assigned points"""
    centroids = np.zeros((k, X.shape[1]))
    for i in range(k):
        centroids[i] = X[labels == i].mean(axis=0)
    return centroids

# Visualize iterations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

centroids = initial_centroids.copy()
colors_map = ['red', 'blue', 'green']

for iteration in range(6):
    ax = axes[iteration]
    
    if iteration == 0:
        # Initial state
        ax.scatter(X_demo[:, 0], X_demo[:, 1], c='gray', s=50, alpha=0.5, edgecolor='black')
        ax.scatter(centroids[:, 0], centroids[:, 1], c=colors_map, 
                   marker='*', s=500, edgecolor='black', linewidth=2, label='Initial Centroids')
        ax.set_title(f'Iteration 0: Initialize Centroids', fontsize=13, fontweight='bold')
    else:
        # Assignment step
        labels = assign_clusters(X_demo, centroids)
        
        # Plot points colored by cluster
        for i in range(k):
            mask = labels == i
            ax.scatter(X_demo[mask, 0], X_demo[mask, 1], c=colors_map[i], 
                       s=50, alpha=0.6, edgecolor='black')
        
        # Plot centroids
        ax.scatter(centroids[:, 0], centroids[:, 1], c=colors_map, 
                   marker='*', s=500, edgecolor='black', linewidth=2)
        
        ax.set_title(f'Iteration {iteration}: Assign & Update', fontsize=13, fontweight='bold')
        
        # Update centroids for next iteration
        centroids = update_centroids(X_demo, labels, k)
    
    ax.set_xlabel('Feature 1', fontsize=11)
    ax.set_ylabel('Feature 2', fontsize=11)
    ax.grid(True, alpha=0.3)

plt.suptitle('K-Means Algorithm: Iterative Process', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\nK-Means iteratively:")
print("1. Assigns points to nearest centroid (Assignment Step)")
print("2. Updates centroids as mean of assigned points (Update Step)")
print("3. Repeats until convergence")

## 2.4 K-Means from Scratch

In [ ]:
class KMeansFromScratch:
    """
    K-Means clustering implementation from scratch
    """
    def __init__(self, n_clusters=3, max_iters=100, tol=1e-4, random_state=None):
        self.n_clusters = n_clusters
        self.max_iters = max_iters
        self.tol = tol
        self.random_state = random_state
        self.centroids = None
        self.labels_ = None
        self.inertia_ = None
        
    def fit(self, X):
        """Fit K-Means to data X"""
        if self.random_state is not None:
            np.random.seed(self.random_state)
        
        # Initialize centroids randomly
        idx = np.random.choice(X.shape[0], self.n_clusters, replace=False)
        self.centroids = X[idx].copy()
        
        for iteration in range(self.max_iters):
            # Assignment step
            distances = cdist(X, self.centroids, 'euclidean')
            self.labels_ = np.argmin(distances, axis=1)
            
            # Update step
            new_centroids = np.zeros_like(self.centroids)
            for i in range(self.n_clusters):
                if np.sum(self.labels_ == i) > 0:
                    new_centroids[i] = X[self.labels_ == i].mean(axis=0)
                else:
                    new_centroids[i] = self.centroids[i]
            
            # Check convergence
            centroid_shift = np.linalg.norm(new_centroids - self.centroids)
            self.centroids = new_centroids
            
            if centroid_shift < self.tol:
                print(f"Converged at iteration {iteration+1}")
                break
        
        # Calculate inertia
        self.inertia_ = 0
        for i in range(self.n_clusters):
            cluster_points = X[self.labels_ == i]
            self.inertia_ += np.sum((cluster_points - self.centroids[i])**2)
        
        return self
    
    def predict(self, X):
        """Predict cluster labels for X"""
        distances = cdist(X, self.centroids, 'euclidean')
        return np.argmin(distances, axis=1)

# Test the implementation
kmeans_scratch = KMeansFromScratch(n_clusters=3, random_state=42)
kmeans_scratch.fit(X_demo)

print(f"\nFinal Inertia: {kmeans_scratch.inertia_:.2f}")
print(f"Final Centroids:\n{kmeans_scratch.centroids}")
print("\n✓ K-Means from scratch implemented successfully!")

---
# Part 3: The Elbow Method for Selecting k

## 3.1 The Problem: Choosing k

K-Means requires us to specify k (number of clusters) in advance. But how do we choose the right k?

**The Elbow Method** is a heuristic approach to find the optimal k.

## 3.2 How the Elbow Method Works

1. Run K-Means for different values of k (e.g., k = 1 to 10)
2. For each k, calculate the **Within-Cluster Sum of Squares (WCSS)** or **Inertia**:
   $$\text{WCSS} = \sum_{i=1}^{K} \sum_{x_j \in C_i} ||x_j - \mu_i||^2$$
3. Plot k vs WCSS
4. Look for the "elbow" - the point where WCSS starts decreasing more slowly

**Intuition**:
- As k increases, WCSS decreases (more clusters = tighter fit)
- At k = n (number of data points), WCSS = 0
- The "elbow" represents a good trade-off between:
  - **Model complexity** (number of clusters)
  - **Model fit** (how well clusters represent data)

## 3.3 Elbow Method Visualization

In [ ]:
# Generate data with clear clusters
X_elbow, y_elbow = make_blobs(n_samples=300, centers=4, n_features=2, 
                               cluster_std=0.7, random_state=42)

# Calculate WCSS for different k values
k_range = range(1, 11)
wcss = []
silhouette_scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_elbow)
    wcss.append(kmeans.inertia_)
    
    # Silhouette score (only for k >= 2)
    if k >= 2:
        silhouette_scores.append(silhouette_score(X_elbow, kmeans.labels_))
    else:
        silhouette_scores.append(0)

# Plot elbow curve
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# WCSS (Elbow) plot
axes[0].plot(k_range, wcss, 'bo-', linewidth=2, markersize=10)
axes[0].axvline(x=4, color='red', linestyle='--', linewidth=2, label='Optimal k=4 (Elbow)')
axes[0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0].set_ylabel('Within-Cluster Sum of Squares (WCSS)', fontsize=12)
axes[0].set_title('Elbow Method: Finding Optimal k', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].legend(fontsize=11)
axes[0].set_xticks(k_range)

# Silhouette score plot
axes[1].plot(k_range, silhouette_scores, 'go-', linewidth=2, markersize=10)
axes[1].axvline(x=4, color='red', linestyle='--', linewidth=2, label='Optimal k=4')
axes[1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score: Validating k', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].legend(fontsize=11)
axes[1].set_xticks(k_range)

plt.tight_layout()
plt.show()

print("\nElbow Method Results:")
print("="*60)
for k, w, s in zip(k_range, wcss, silhouette_scores):
    print(f"k={k:2d}  WCSS={w:8.2f}  Silhouette={s:.4f}")

print("\n→ The 'elbow' occurs at k=4")
print("→ Silhouette score is also highest at k=4")
print("→ Optimal number of clusters: k=4")

## 3.4 Understanding the Elbow

**What to look for:**
- The "elbow" is the point where the curve bends sharply
- Before the elbow: Adding clusters significantly reduces WCSS
- After the elbow: Adding clusters provides diminishing returns

**Limitations:**
- The elbow may not always be clear
- Subjective interpretation
- Works best when clusters are well-separated

**Alternative Metrics:**
- **Silhouette Score**: Measures how similar a point is to its own cluster vs other clusters
  - Range: [-1, 1]
  - Higher is better
- **Davies-Bouldin Index**: Ratio of within-cluster to between-cluster distances
  - Lower is better

---
# Part 4: Practical Example - Customer Segmentation

## 4.1 Problem Setup

Let's apply K-Means to a realistic customer segmentation problem using synthetic data.

In [ ]:
# Generate synthetic customer data
np.random.seed(42)
n_customers = 400

# Features: Annual Income (k$), Spending Score (1-100)
# Create 5 distinct customer segments
segment1 = np.random.randn(80, 2) * [5, 8] + [25, 25]   # Low income, low spending
segment2 = np.random.randn(80, 2) * [5, 8] + [25, 75]   # Low income, high spending
segment3 = np.random.randn(80, 2) * [5, 8] + [50, 50]   # Medium income, medium spending
segment4 = np.random.randn(80, 2) * [5, 8] + [75, 25]   # High income, low spending
segment5 = np.random.randn(80, 2) * [5, 8] + [75, 75]   # High income, high spending

X_customers = np.vstack([segment1, segment2, segment3, segment4, segment5])

# Create DataFrame
df_customers = pd.DataFrame(X_customers, columns=['Annual_Income_k$', 'Spending_Score'])

print("Customer Data:")
print("="*60)
print(df_customers.describe())
print(f"\nTotal customers: {len(df_customers)}")

# Visualize raw data
plt.figure(figsize=(10, 6))
plt.scatter(df_customers['Annual_Income_k$'], df_customers['Spending_Score'], 
            c='darkblue', s=60, alpha=0.6, edgecolor='black')
plt.xlabel('Annual Income (k$)', fontsize=12)
plt.ylabel('Spending Score (1-100)', fontsize=12)
plt.title('Customer Data: Annual Income vs Spending Score', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4.2 Apply Elbow Method

In [ ]:
# Standardize features
scaler = StandardScaler()
X_customers_scaled = scaler.fit_transform(X_customers)

# Elbow method
k_range = range(1, 11)
wcss_customers = []
silhouette_customers = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_customers_scaled)
    wcss_customers.append(kmeans.inertia_)
    
    if k >= 2:
        silhouette_customers.append(silhouette_score(X_customers_scaled, kmeans.labels_))
    else:
        silhouette_customers.append(0)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].plot(k_range, wcss_customers, 'bo-', linewidth=2, markersize=10)
axes[0].axvline(x=5, color='red', linestyle='--', linewidth=2, label='Optimal k=5')
axes[0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0].set_ylabel('WCSS', fontsize=12)
axes[0].set_title('Elbow Method: Customer Segmentation', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].legend(fontsize=11)
axes[0].set_xticks(k_range)

axes[1].plot(k_range, silhouette_customers, 'go-', linewidth=2, markersize=10)
axes[1].axvline(x=5, color='red', linestyle='--', linewidth=2, label='Optimal k=5')
axes[1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score: Customer Segmentation', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].legend(fontsize=11)
axes[1].set_xticks(k_range)

plt.tight_layout()
plt.show()

print("\n→ Optimal k = 5 customer segments")

## 4.3 Final Clustering with k=5

In [ ]:
# Apply K-Means with k=5
kmeans_final = KMeans(n_clusters=5, random_state=42, n_init=10)
df_customers['Cluster'] = kmeans_final.fit_predict(X_customers_scaled)

# Get centroids in original scale
centroids_scaled = kmeans_final.cluster_centers_
centroids_original = scaler.inverse_transform(centroids_scaled)

# Visualize clusters
plt.figure(figsize=(12, 8))

colors = ['red', 'blue', 'green', 'orange', 'purple']
segment_names = ['Low Income\nLow Spending', 'Low Income\nHigh Spending', 
                 'Medium Income\nMedium Spending', 'High Income\nLow Spending', 
                 'High Income\nHigh Spending']

for i in range(5):
    cluster_data = df_customers[df_customers['Cluster'] == i]
    plt.scatter(cluster_data['Annual_Income_k$'], cluster_data['Spending_Score'], 
                c=colors[i], label=f'Segment {i+1}: {segment_names[i]}', 
                s=80, alpha=0.6, edgecolor='black')

# Plot centroids
plt.scatter(centroids_original[:, 0], centroids_original[:, 1], 
            c='black', marker='*', s=500, edgecolor='white', linewidth=2, 
            label='Centroids', zorder=10)

plt.xlabel('Annual Income (k$)', fontsize=13)
plt.ylabel('Spending Score (1-100)', fontsize=13)
plt.title('Customer Segmentation: 5 Distinct Segments', fontsize=15, fontweight='bold')
plt.legend(fontsize=10, loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Cluster statistics
print("\nCluster Statistics:")
print("="*80)
for i in range(5):
    cluster_data = df_customers[df_customers['Cluster'] == i]
    print(f"\nSegment {i+1}: {segment_names[i]}")
    print(f"  Size: {len(cluster_data)} customers ({len(cluster_data)/len(df_customers)*100:.1f}%)")
    print(f"  Avg Income: ${cluster_data['Annual_Income_k$'].mean():.1f}k")
    print(f"  Avg Spending Score: {cluster_data['Spending_Score'].mean():.1f}")
    print(f"  Centroid: Income=${centroids_original[i, 0]:.1f}k, Score={centroids_original[i, 1]:.1f}")

## 4.4 Business Insights

Based on the customer segmentation:

**Segment 1: Low Income, Low Spending**
- Strategy: Budget-friendly products, discounts, loyalty programs

**Segment 2: Low Income, High Spending**
- Strategy: Credit options, installment plans, value deals

**Segment 3: Medium Income, Medium Spending**
- Strategy: Balanced product mix, seasonal promotions

**Segment 4: High Income, Low Spending**
- Strategy: Premium products, exclusive offers, personalized service

**Segment 5: High Income, High Spending**
- Strategy: VIP treatment, luxury products, premium experiences

---
# Part 5: Real Dataset Example - Iris Dataset

## 5.1 Load and Explore Data

In [ ]:
# Load Iris dataset
iris = load_iris()
X_iris = iris.data
y_iris = iris.target
feature_names = iris.feature_names

df_iris = pd.DataFrame(X_iris, columns=feature_names)
df_iris['species'] = iris.target_names[y_iris]

print("Iris Dataset:")
print("="*60)
print(df_iris.head(10))
print(f"\nShape: {X_iris.shape}")
print(f"Features: {feature_names}")
print(f"Species: {iris.target_names}")

## 5.2 Apply K-Means (Unsupervised)

In [ ]:
# Standardize
scaler_iris = StandardScaler()
X_iris_scaled = scaler_iris.fit_transform(X_iris)

# Elbow method
k_range = range(1, 11)
wcss_iris = []
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_iris_scaled)
    wcss_iris.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(k_range, wcss_iris, 'bo-', linewidth=2, markersize=10)
plt.axvline(x=3, color='red', linestyle='--', linewidth=2, label='Optimal k=3')
plt.xlabel('Number of Clusters (k)', fontsize=12)
plt.ylabel('WCSS', fontsize=12)
plt.title('Elbow Method: Iris Dataset', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.xticks(k_range)
plt.tight_layout()
plt.show()

# Apply K-Means with k=3
kmeans_iris = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters_iris = kmeans_iris.fit_predict(X_iris_scaled)

print(f"\nOptimal k = 3 (matches the true number of species!)")

## 5.3 Compare Clusters with True Labels

In [ ]:
# Visualize: Petal Length vs Petal Width
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# True labels
colors_true = ['red', 'blue', 'green']
for i, species in enumerate(iris.target_names):
    mask = y_iris == i
    axes[0].scatter(X_iris[mask, 2], X_iris[mask, 3], c=colors_true[i], 
                    label=species, s=80, alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Petal Length (cm)', fontsize=12)
axes[0].set_ylabel('Petal Width (cm)', fontsize=12)
axes[0].set_title('True Species Labels', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# K-Means clusters
colors_cluster = ['orange', 'purple', 'cyan']
for i in range(3):
    mask = clusters_iris == i
    axes[1].scatter(X_iris[mask, 2], X_iris[mask, 3], c=colors_cluster[i], 
                    label=f'Cluster {i}', s=80, alpha=0.7, edgecolor='black')

# Plot centroids
centroids_iris = scaler_iris.inverse_transform(kmeans_iris.cluster_centers_)
axes[1].scatter(centroids_iris[:, 2], centroids_iris[:, 3], 
                c='black', marker='*', s=500, edgecolor='white', linewidth=2, 
                label='Centroids', zorder=10)

axes[1].set_xlabel('Petal Length (cm)', fontsize=12)
axes[1].set_ylabel('Petal Width (cm)', fontsize=12)
axes[1].set_title('K-Means Clusters (k=3)', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Accuracy (with best label mapping)
from scipy.stats import mode
labels_mapped = np.zeros_like(clusters_iris)
for i in range(3):
    mask = clusters_iris == i
    labels_mapped[mask] = mode(y_iris[mask], keepdims=True)[0][0]

accuracy = np.mean(labels_mapped == y_iris)
print(f"\nClustering Accuracy (with best mapping): {accuracy*100:.1f}%")
print("\n→ K-Means successfully discovered the 3 species without labels!")

---
# Part 6: K-Means Limitations and Best Practices

## 6.1 Limitations

**1. Requires k to be specified**
- Solution: Use elbow method, silhouette analysis

**2. Sensitive to initialization**
- Solution: Run multiple times with different initializations (`n_init` parameter)

**3. Assumes spherical clusters**
- Problem: Fails with elongated or irregular shapes
- Solution: Consider DBSCAN, Gaussian Mixture Models

**4. Sensitive to outliers**
- Solution: Remove outliers, use K-medoids (more robust)

**5. Sensitive to scale**
- Solution: Always standardize features

## 6.2 Best Practices

1. **Standardize features** before clustering
2. **Use multiple initializations** (`n_init=10` or more)
3. **Try different k values** and use elbow method
4. **Validate with multiple metrics** (silhouette, Davies-Bouldin)
5. **Visualize results** when possible
6. **Consider domain knowledge** when interpreting clusters
7. **Check for outliers** before clustering

## 6.3 When to Use K-Means

**Good for:**
- Large datasets (fast and scalable)
- Spherical, well-separated clusters
- When k is known or can be estimated
- Exploratory data analysis

**Not good for:**
- Non-spherical clusters
- Clusters with very different sizes
- Noisy data with many outliers
- When cluster density varies significantly

---
# Part 7: Summary and Key Takeaways

## What We Learned

1. **Cluster Analysis** is unsupervised learning for discovering patterns in unlabeled data

2. **K-Means Algorithm**:
   - Iteratively assigns points to nearest centroid
   - Updates centroids as mean of assigned points
   - Minimizes within-cluster sum of squares (WCSS)
   - Simple, fast, and widely used

3. **Elbow Method**:
   - Plot k vs WCSS
   - Look for the "elbow" point
   - Represents optimal trade-off between complexity and fit

4. **Practical Applications**:
   - Customer segmentation
   - Image segmentation
   - Document clustering
   - Anomaly detection

5. **Best Practices**:
   - Standardize features
   - Use multiple initializations
   - Validate with multiple metrics
   - Visualize and interpret results

## Further Reading

- K-Means++: Improved initialization method
- Mini-Batch K-Means: Faster variant for large datasets
- DBSCAN: Density-based clustering (handles non-spherical clusters)
- Hierarchical Clustering: Creates tree of clusters
- Gaussian Mixture Models: Probabilistic clustering

---

**End of Tutorial**

Thank you for learning K-Means clustering! Practice with different datasets to master the technique.

---
# Part 8: Hierarchical Clustering

Hierarchical clustering builds a tree of clusters called a **dendrogram**.

## 8.1 Introduction

**Two Approaches:**
- **Agglomerative (AGNES)**: Bottom-up, start with individual points, merge closest
- **Divisive (DIANA)**: Top-down, start with one cluster, divide based on criterion

**Advantages:**
- No need to specify k in advance
- Produces a dendrogram (visual hierarchy)
- Can capture nested clusters

**Disadvantages:**
- Computationally expensive: O(n³) or O(n² log n)
- Cannot undo merges/splits
- Sensitive to noise and outliers

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

# Generate sample data
np.random.seed(42)
X_hier = np.vstack([
    np.random.randn(30, 2) + [2, 2],
    np.random.randn(30, 2) + [8, 2],
    np.random.randn(30, 2) + [5, 8]
])

print(f"Generated {len(X_hier)} points for hierarchical clustering")

## 8.2 Understanding Dendrograms

A dendrogram shows:
- Each vertical line represents a cluster
- Height shows the distance at which clusters merge
- Cutting at different heights gives different numbers of clusters

In [ ]:
# Create dendrogram
plt.figure(figsize=(12, 6))
linkage_matrix = linkage(X_hier, method='ward')
dendrogram(linkage_matrix)
plt.title('Hierarchical Clustering Dendrogram (Ward Linkage)', fontsize=14, fontweight='bold')
plt.xlabel('Sample Index')
plt.ylabel('Distance')
plt.axhline(y=15, color='r', linestyle='--', label='Cut at height 15 (3 clusters)')
plt.legend()
plt.tight_layout()
plt.show()

print("How to read the dendrogram:")
print("• Each vertical line represents a cluster")
print("• Height shows the distance at which clusters merge")
print("• Horizontal red line shows where we cut to get 3 clusters")
print("• Cutting at different heights gives different numbers of clusters")

## 8.3 Linkage Methods

Different ways to measure distance between clusters:

1. **Single Linkage**: Minimum distance between any two points
   - dist(Ci, Cj) = min{dist(xp, xq) : xp ∈ Ci, xq ∈ Cj}
   - Tends to create elongated clusters (chain effect)

2. **Complete Linkage**: Maximum distance between any two points
   - dist(Ci, Cj) = max{dist(xp, xq) : xp ∈ Ci, xq ∈ Cj}
   - Creates compact, spherical clusters

3. **Average Linkage**: Average distance between all pairs
   - dist(Ci, Cj) = (1/(Ni×Nj)) × Σ dist(xp, xq)
   - Balanced approach

4. **Ward Linkage**: Minimizes within-cluster variance
   - Most recommended for general use

In [ ]:
# Compare linkage methods
linkage_methods = ['single', 'complete', 'average', 'ward']
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.ravel()

for idx, method in enumerate(linkage_methods):
    agg = AgglomerativeClustering(n_clusters=3, linkage=method)
    labels = agg.fit_predict(X_hier)
    
    axes[idx].scatter(X_hier[:, 0], X_hier[:, 1], c=labels, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
    axes[idx].set_title(f'{method.capitalize()} Linkage', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Feature 1')
    axes[idx].set_ylabel('Feature 2')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Linkage Method Characteristics:")
print("• Single: Tends to create elongated clusters (chain effect)")
print("• Complete: Creates compact, spherical clusters")
print("• Average: Balanced approach")
print("• Ward: Minimizes variance (recommended)")

## 8.4 Dendrogram Comparison

Let's compare dendrograms for different linkage methods.

In [ ]:
# Compare dendrograms
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for idx, method in enumerate(linkage_methods):
    linkage_matrix = linkage(X_hier, method=method)
    dendrogram(linkage_matrix, ax=axes[idx])
    axes[idx].set_title(f'{method.capitalize()} Linkage Dendrogram', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Sample Index')
    axes[idx].set_ylabel('Distance')

plt.tight_layout()
plt.show()

print("Notice how different linkage methods produce different tree structures")


## 8.5 Choosing Number of Clusters

Use silhouette score to find optimal number of clusters.

In [ ]:
# Find optimal k for hierarchical clustering
k_range = range(2, 8)
silhouette_scores_hier = []

for k in k_range:
    agg = AgglomerativeClustering(n_clusters=k, linkage='ward')
    labels = agg.fit_predict(X_hier)
    score = silhouette_score(X_hier, labels)
    silhouette_scores_hier.append(score)

plt.figure(figsize=(10, 5))
plt.plot(k_range, silhouette_scores_hier, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Number of Clusters', fontsize=12)
plt.ylabel('Silhouette Score', fontsize=12)
plt.title('Hierarchical Clustering: Silhouette Score vs k', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

best_k_hier = k_range[np.argmax(silhouette_scores_hier)]
print(f"Optimal k: {best_k_hier}")


## 8.6 Hierarchical Clustering from Scratch

Let's implement agglomerative clustering from scratch.

In [ ]:
class HierarchicalClusteringFromScratch:
    def __init__(self, n_clusters=3, linkage='single'):
        self.n_clusters = n_clusters
        self.linkage = linkage
        self.labels_ = None
    
    def fit_predict(self, X):
        n_samples = len(X)
        # Initialize: each point is its own cluster
        clusters = [[i] for i in range(n_samples)]
        
        # Compute initial distance matrix
        from scipy.spatial.distance import pdist, squareform
        dist_matrix = squareform(pdist(X, metric='euclidean'))
        
        # Merge until we have desired number of clusters
        while len(clusters) > self.n_clusters:
            # Find closest pair of clusters
            min_dist = float('inf')
            merge_i, merge_j = 0, 1
            
            for i in range(len(clusters)):
                for j in range(i + 1, len(clusters)):
                    dist = self._cluster_distance(clusters[i], clusters[j], dist_matrix)
                    if dist < min_dist:
                        min_dist = dist
                        merge_i, merge_j = i, j
            
            # Merge the two closest clusters
            clusters[merge_i].extend(clusters[merge_j])
            del clusters[merge_j]
        
        # Create labels array
        labels = np.zeros(n_samples, dtype=int)
        for cluster_id, cluster_points in enumerate(clusters):
            for point_idx in cluster_points:
                labels[point_idx] = cluster_id
        
        self.labels_ = labels
        return labels
    
    def _cluster_distance(self, cluster1, cluster2, dist_matrix):
        if self.linkage == 'single':
            # Minimum distance
            return min(dist_matrix[i, j] for i in cluster1 for j in cluster2)
        elif self.linkage == 'complete':
            # Maximum distance
            return max(dist_matrix[i, j] for i in cluster1 for j in cluster2)
        elif self.linkage == 'average':
            # Average distance
            distances = [dist_matrix[i, j] for i in cluster1 for j in cluster2]
            return np.mean(distances)
        else:
            raise ValueError(f"Unknown linkage: {self.linkage}")

# Test our implementation
hier_scratch = HierarchicalClusteringFromScratch(n_clusters=3, linkage='average')
labels_scratch = hier_scratch.fit_predict(X_hier)

# Compare with scikit-learn
hier_sklearn = AgglomerativeClustering(n_clusters=3, linkage='average')
labels_sklearn = hier_sklearn.fit_predict(X_hier)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].scatter(X_hier[:, 0], X_hier[:, 1], c=labels_scratch, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[0].set_title('From Scratch', fontweight='bold')
axes[1].scatter(X_hier[:, 0], X_hier[:, 1], c=labels_sklearn, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[1].set_title('Scikit-learn', fontweight='bold')
plt.tight_layout()
plt.show()

print("From-scratch implementation validated!")


## 8.7 Summary: Hierarchical Clustering

**When to use:**
- Need hierarchy of clusters
- Don't know k in advance
- Small to medium datasets

**Linkage recommendations:**
- **Ward**: Best for most cases
- **Complete**: For compact clusters
- **Average**: Balanced approach
- **Single**: Avoid unless you need chain-like clusters

---
# Part 9: DBSCAN (Density-Based Clustering)

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) finds clusters based on density.

## 9.1 Key Concepts

**Parameters:**
- **ε (epsilon)**: Maximum distance between two points to be considered neighbors
- **MinPts**: Minimum number of points to form a dense region

**Point Types:**
- **Core point**: Has at least MinPts neighbors within ε
- **Border point**: Within ε of a core point but has fewer than MinPts neighbors
- **Noise point**: Neither core nor border

**Advantages:**
- Finds arbitrary-shaped clusters
- Identifies outliers as noise
- No need to specify number of clusters

**Disadvantages:**
- Sensitive to ε and MinPts parameters
- Struggles with varying densities

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons, make_circles

# Create datasets where K-Means fails
X_moons, _ = make_moons(n_samples=200, noise=0.05, random_state=42)
X_circles, _ = make_circles(n_samples=200, noise=0.05, factor=0.5, random_state=42)

print("Created challenging datasets for DBSCAN demonstration")

## 9.2 DBSCAN vs K-Means

Let's see how DBSCAN handles non-spherical clusters where K-Means fails.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

datasets = [(X_moons, 'Moons'), (X_circles, 'Circles')]

for row, (X, name) in enumerate(datasets):
    # Original data
    axes[row, 0].scatter(X[:, 0], X[:, 1], s=50, alpha=0.6)
    axes[row, 0].set_title(f'{name} Dataset', fontweight='bold')
    
    # K-Means
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    labels_km = kmeans.fit_predict(X)
    axes[row, 1].scatter(X[:, 0], X[:, 1], c=labels_km, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
    axes[row, 1].set_title(f'K-Means (Fails)', fontweight='bold')
    
    # DBSCAN
    dbscan = DBSCAN(eps=0.3, min_samples=5)
    labels_db = dbscan.fit_predict(X)
    axes[row, 2].scatter(X[:, 0], X[:, 1], c=labels_db, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
    # Mark noise points
    noise = X[labels_db == -1]
    axes[row, 2].scatter(noise[:, 0], noise[:, 1], c='red', marker='x', s=100, label='Noise')
    axes[row, 2].set_title(f'DBSCAN (Succeeds)', fontweight='bold')
    axes[row, 2].legend()

plt.tight_layout()
plt.show()

print("Key Observations:")
print("• K-Means assumes spherical clusters and fails on non-convex shapes")
print("• DBSCAN successfully identifies the true structure")
print("• DBSCAN identifies outliers as noise (red X markers)")

## 9.3 DBSCAN Algorithm Step-by-Step

**Algorithm:**
1. For each point p:
   - Find all points within ε distance (neighbors)
   - If neighbors >= MinPts, p is a core point
2. For each core point:
   - Create a cluster with all density-reachable points
3. Assign border points to nearby clusters
4. Mark remaining points as noise

**Time Complexity:** O(n log n) with spatial indexing

In [ ]:
# Visualize DBSCAN concepts
from sklearn.neighbors import NearestNeighbors

# Create simple dataset
np.random.seed(42)
X_demo = np.random.randn(50, 2)

# Run DBSCAN
dbscan_demo = DBSCAN(eps=0.5, min_samples=5)
labels_demo = dbscan_demo.fit_predict(X_demo)

# Identify point types
core_samples = np.zeros(len(X_demo), dtype=bool)
core_samples[dbscan_demo.core_sample_indices_] = True
noise = labels_demo == -1
border = ~core_samples & ~noise

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Clusters
axes[0].scatter(X_demo[:, 0], X_demo[:, 1], c=labels_demo, cmap='viridis', s=100, alpha=0.6, edgecolors='k')
axes[0].scatter(X_demo[noise, 0], X_demo[noise, 1], c='red', marker='x', s=200, linewidths=3, label='Noise')
axes[0].set_title('DBSCAN Clusters', fontweight='bold')
axes[0].legend()

# Point types
axes[1].scatter(X_demo[core_samples, 0], X_demo[core_samples, 1], c='blue', s=100, label='Core', edgecolors='k')
axes[1].scatter(X_demo[border, 0], X_demo[border, 1], c='orange', s=100, marker='s', label='Border', edgecolors='k')
axes[1].scatter(X_demo[noise, 0], X_demo[noise, 1], c='red', s=100, marker='x', label='Noise', linewidths=3)
axes[1].set_title('Point Classification', fontweight='bold')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Core points: {core_samples.sum()}")
print(f"Border points: {border.sum()}")
print(f"Noise points: {noise.sum()}")


## 9.4 Parameter Selection: K-Distance Graph

The **K-distance graph** helps choose ε:
1. For each point, find distance to k-th nearest neighbor
2. Sort distances in ascending order
3. Plot the sorted distances
4. Look for the "elbow" - sharp increase indicates good ε

In [ ]:
# K-distance graph for parameter selection
k = 5  # MinPts
nbrs = NearestNeighbors(n_neighbors=k).fit(X_moons)
distances, indices = nbrs.kneighbors(X_moons)
distances = np.sort(distances[:, k-1], axis=0)

plt.figure(figsize=(10, 5))
plt.plot(distances, linewidth=2)
plt.ylabel('K-th Nearest Neighbor Distance', fontsize=12)
plt.xlabel('Points sorted by distance', fontsize=12)
plt.title('K-Distance Graph (k=5)', fontsize=14, fontweight='bold')
plt.axhline(y=0.3, color='r', linestyle='--', label='Suggested ε = 0.3')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

print("The elbow point suggests ε ≈ 0.3")


## 9.5 Effect of Parameters

Let's see how different ε values affect clustering.

In [ ]:
# Try different epsilon values
eps_values = [0.1, 0.2, 0.3, 0.4]
fig, axes = plt.subplots(1, 4, figsize=(18, 4))

for idx, eps in enumerate(eps_values):
    dbscan = DBSCAN(eps=eps, min_samples=5)
    labels = dbscan.fit_predict(X_moons)
    
    axes[idx].scatter(X_moons[:, 0], X_moons[:, 1], c=labels, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
    noise = X_moons[labels == -1]
    axes[idx].scatter(noise[:, 0], noise[:, 1], c='red', marker='x', s=100, linewidths=2)
    axes[idx].set_title(f'ε = {eps}', fontweight='bold')
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)
    axes[idx].set_xlabel(f'{n_clusters} clusters, {n_noise} noise')

plt.tight_layout()
plt.show()

print("Observations:")
print("• Too small ε: Many noise points, fragmented clusters")
print("• Optimal ε: Clean separation, few noise points")
print("• Too large ε: Clusters merge together")


## 9.6 DBSCAN from Scratch

In [ ]:
class DBSCANFromScratch:
    def __init__(self, eps=0.5, min_samples=5):
        self.eps = eps
        self.min_samples = min_samples
        self.labels_ = None
    
    def fit_predict(self, X):
        n_samples = len(X)
        labels = np.full(n_samples, -1)  # -1 means noise
        cluster_id = 0
        
        # Compute distance matrix
        from scipy.spatial.distance import cdist
        distances = cdist(X, X, metric='euclidean')
        
        # Find neighbors for each point
        neighbors = [np.where(distances[i] <= self.eps)[0] for i in range(n_samples)]
        
        # Identify core points
        core_points = [i for i in range(n_samples) if len(neighbors[i]) >= self.min_samples]
        
        # Expand clusters from core points
        for point in core_points:
            if labels[point] != -1:
                continue  # Already assigned
            
            # Start new cluster
            labels[point] = cluster_id
            seeds = list(neighbors[point])
            
            # Expand cluster
            i = 0
            while i < len(seeds):
                q = seeds[i]
                
                if labels[q] == -1:
                    labels[q] = cluster_id
                
                if labels[q] != -1:
                    i += 1
                    continue
                
                labels[q] = cluster_id
                
                # If q is core point, add its neighbors
                if len(neighbors[q]) >= self.min_samples:
                    seeds.extend(neighbors[q])
                
                i += 1
            
            cluster_id += 1
        
        self.labels_ = labels
        return labels

# Test implementation
dbscan_scratch = DBSCANFromScratch(eps=0.3, min_samples=5)
labels_scratch = dbscan_scratch.fit_predict(X_moons)

dbscan_sklearn = DBSCAN(eps=0.3, min_samples=5)
labels_sklearn = dbscan_sklearn.fit_predict(X_moons)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].scatter(X_moons[:, 0], X_moons[:, 1], c=labels_scratch, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[0].set_title('From Scratch', fontweight='bold')
axes[1].scatter(X_moons[:, 0], X_moons[:, 1], c=labels_sklearn, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[1].set_title('Scikit-learn', fontweight='bold')
plt.tight_layout()
plt.show()

print("From-scratch DBSCAN validated!")


---
# Part 10: Spectral Clustering

Spectral clustering uses graph theory and eigenvalues to find clusters.

## 10.1 Core Concepts

**Algorithm:**
1. Construct similarity graph
2. Compute graph Laplacian
3. Find eigenvectors
4. Apply K-Means on eigenvectors

**Advantages:**
- Finds non-convex clusters
- Works well with graph-structured data
- Can capture complex cluster shapes

**Disadvantages:**
- Computationally expensive
- Requires choosing number of clusters
- Sensitive to similarity metric

In [ ]:
from sklearn.cluster import SpectralClustering

# Spectral vs K-Means on concentric circles
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Original
axes[0].scatter(X_circles[:, 0], X_circles[:, 1], s=50, alpha=0.6)
axes[0].set_title('Original Data (Concentric Circles)', fontweight='bold')

# K-Means
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
labels_km = kmeans.fit_predict(X_circles)
axes[1].scatter(X_circles[:, 0], X_circles[:, 1], c=labels_km, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[1].set_title('K-Means (Fails)', fontweight='bold')

# Spectral
spectral = SpectralClustering(n_clusters=2, affinity='nearest_neighbors', random_state=42)
labels_sp = spectral.fit_predict(X_circles)
axes[2].scatter(X_circles[:, 0], X_circles[:, 1], c=labels_sp, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[2].set_title('Spectral Clustering (Succeeds)', fontweight='bold')

plt.tight_layout()
plt.show()

print("Spectral clustering successfully separates concentric circles!")

## 10.2 Mathematical Foundation

Spectral clustering uses eigenvalues and eigenvectors of the graph Laplacian.

**Steps:**
1. **Construct affinity matrix W**: Similarity between points
2. **Compute degree matrix D**: D_ii = Σ W_ij
3. **Compute Laplacian**: L = D - W (unnormalized) or L_norm = D^(-1/2) L D^(-1/2) (normalized)
4. **Find eigenvectors**: Compute k smallest eigenvectors of L
5. **Apply K-Means**: Cluster rows of eigenvector matrix

In [ ]:
# Demonstrate spectral clustering step-by-step
from sklearn.metrics.pairwise import rbf_kernel
from scipy.linalg import eigh

# Use circles dataset
X_demo = X_circles[:50]  # Smaller for visualization

# Step 1: Affinity matrix
gamma = 1.0
W = rbf_kernel(X_demo, gamma=gamma)

# Step 2: Degree matrix
D = np.diag(W.sum(axis=1))

# Step 3: Normalized Laplacian
D_inv_sqrt = np.diag(1.0 / np.sqrt(np.diag(D)))
L_norm = np.eye(len(X_demo)) - D_inv_sqrt @ W @ D_inv_sqrt

# Step 4: Eigendecomposition
eigenvalues, eigenvectors = eigh(L_norm)

# Visualize the process
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Original data
axes[0, 0].scatter(X_demo[:, 0], X_demo[:, 1], s=100, alpha=0.6, edgecolors='k')
axes[0, 0].set_title('1. Original Data', fontweight='bold')

# Affinity matrix
im1 = axes[0, 1].imshow(W, cmap='viridis')
axes[0, 1].set_title('2. Affinity Matrix W', fontweight='bold')
plt.colorbar(im1, ax=axes[0, 1])

# Laplacian
im2 = axes[0, 2].imshow(L_norm, cmap='RdBu')
axes[0, 2].set_title('3. Normalized Laplacian', fontweight='bold')
plt.colorbar(im2, ax=axes[0, 2])

# Eigenvalue spectrum
axes[1, 0].plot(eigenvalues[:20], 'bo-', linewidth=2, markersize=8)
axes[1, 0].set_title('4. Eigenvalue Spectrum', fontweight='bold')
axes[1, 0].set_xlabel('Index')
axes[1, 0].set_ylabel('Eigenvalue')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axvline(x=2, color='r', linestyle='--', label='k=2')
axes[1, 0].legend()

# Eigenvector space
X_embedded = eigenvectors[:, :2]
axes[1, 1].scatter(X_embedded[:, 0], X_embedded[:, 1], s=100, alpha=0.6, edgecolors='k')
axes[1, 1].set_title('5. Eigenvector Space', fontweight='bold')
axes[1, 1].set_xlabel('1st eigenvector')
axes[1, 1].set_ylabel('2nd eigenvector')

# Final clustering
from sklearn.cluster import KMeans
kmeans_spectral = KMeans(n_clusters=2, random_state=42, n_init=10)
labels_spectral = kmeans_spectral.fit_predict(X_embedded)
axes[1, 2].scatter(X_demo[:, 0], X_demo[:, 1], c=labels_spectral, cmap='viridis', s=100, alpha=0.6, edgecolors='k')
axes[1, 2].set_title('6. Final Clusters', fontweight='bold')

plt.tight_layout()
plt.show()

print("Spectral clustering transforms the data into eigenvector space where K-Means works!")


## 10.3 Eigengap Heuristic

The **eigengap** helps choose the number of clusters k.

**Rule:** Choose k where there's a large gap between λ_k and λ_(k+1)

In [ ]:
# Eigengap heuristic demonstration
W_full = rbf_kernel(X_circles, gamma=1.0)
D_full = np.diag(W_full.sum(axis=1))
D_inv_sqrt_full = np.diag(1.0 / np.sqrt(np.diag(D_full)))
L_norm_full = np.eye(len(X_circles)) - D_inv_sqrt_full @ W_full @ D_inv_sqrt_full
eigenvalues_full, _ = eigh(L_norm_full)

# Plot eigenvalues and eigengaps
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Eigenvalue spectrum
axes[0].plot(eigenvalues_full[:15], 'bo-', linewidth=2, markersize=8)
axes[0].set_title('Eigenvalue Spectrum', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Index')
axes[0].set_ylabel('Eigenvalue')
axes[0].grid(True, alpha=0.3)

# Eigengap
eigengaps = np.diff(eigenvalues_full[:15])
axes[1].bar(range(1, len(eigengaps)+1), eigengaps)
axes[1].set_title('Eigengap (λ_k - λ_(k-1))', fontsize=14, fontweight='bold')
axes[1].set_xlabel('k')
axes[1].set_ylabel('Gap')
axes[1].grid(True, alpha=0.3)

# Highlight largest gap
max_gap_idx = np.argmax(eigengaps) + 1
axes[1].bar(max_gap_idx, eigengaps[max_gap_idx-1], color='red', label=f'Largest gap at k={max_gap_idx}')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Eigengap heuristic suggests k = {max_gap_idx}")


## 10.4 Affinity Matrix Construction

Different ways to build the affinity matrix:
1. **RBF kernel**: W_ij = exp(-γ||x_i - x_j||²)
2. **K-nearest neighbors**: Connect only k nearest neighbors
3. **ε-neighborhood**: Connect if distance < ε

In [ ]:
# Compare affinity types
from sklearn.neighbors import kneighbors_graph

fig, axes = plt.subplots(2, 2, figsize=(12, 12))

# RBF kernel
spectral_rbf = SpectralClustering(n_clusters=2, affinity='rbf', gamma=1.0, random_state=42)
labels_rbf = spectral_rbf.fit_predict(X_circles)
axes[0, 0].scatter(X_circles[:, 0], X_circles[:, 1], c=labels_rbf, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[0, 0].set_title('RBF Kernel', fontweight='bold')

# Nearest neighbors
spectral_nn = SpectralClustering(n_clusters=2, affinity='nearest_neighbors', n_neighbors=10, random_state=42)
labels_nn = spectral_nn.fit_predict(X_circles)
axes[0, 1].scatter(X_circles[:, 0], X_circles[:, 1], c=labels_nn, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[0, 1].set_title('Nearest Neighbors (k=10)', fontweight='bold')

# Different gamma for RBF
spectral_rbf2 = SpectralClustering(n_clusters=2, affinity='rbf', gamma=0.1, random_state=42)
labels_rbf2 = spectral_rbf2.fit_predict(X_circles)
axes[1, 0].scatter(X_circles[:, 0], X_circles[:, 1], c=labels_rbf2, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[1, 0].set_title('RBF Kernel (γ=0.1)', fontweight='bold')

# Different k for NN
spectral_nn2 = SpectralClustering(n_clusters=2, affinity='nearest_neighbors', n_neighbors=5, random_state=42)
labels_nn2 = spectral_nn2.fit_predict(X_circles)
axes[1, 1].scatter(X_circles[:, 0], X_circles[:, 1], c=labels_nn2, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[1, 1].set_title('Nearest Neighbors (k=5)', fontweight='bold')

plt.tight_layout()
plt.show()

print("Affinity matrix construction affects clustering results")


## 10.5 Spectral Clustering from Scratch

In [ ]:
class SpectralClusteringFromScratch:
    def __init__(self, n_clusters=2, gamma=1.0):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.labels_ = None
    
    def fit_predict(self, X):
        # Step 1: Compute affinity matrix (RBF kernel)
        W = rbf_kernel(X, gamma=self.gamma)
        
        # Step 2: Compute degree matrix
        D = np.diag(W.sum(axis=1))
        
        # Step 3: Normalized Laplacian
        D_inv_sqrt = np.diag(1.0 / np.sqrt(np.diag(D) + 1e-10))  # Add small value for stability
        L_norm = np.eye(len(X)) - D_inv_sqrt @ W @ D_inv_sqrt
        
        # Step 4: Eigendecomposition
        eigenvalues, eigenvectors = eigh(L_norm)
        
        # Step 5: Select k smallest eigenvectors
        X_embedded = eigenvectors[:, :self.n_clusters]
        
        # Step 6: Normalize rows
        X_embedded = X_embedded / (np.linalg.norm(X_embedded, axis=1, keepdims=True) + 1e-10)
        
        # Step 7: Apply K-Means
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=42, n_init=10)
        self.labels_ = kmeans.fit_predict(X_embedded)
        
        return self.labels_

# Test implementation
spectral_scratch = SpectralClusteringFromScratch(n_clusters=2, gamma=1.0)
labels_scratch = spectral_scratch.fit_predict(X_circles)

spectral_sklearn = SpectralClustering(n_clusters=2, affinity='rbf', gamma=1.0, random_state=42)
labels_sklearn = spectral_sklearn.fit_predict(X_circles)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].scatter(X_circles[:, 0], X_circles[:, 1], c=labels_scratch, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[0].set_title('From Scratch', fontweight='bold')
axes[1].scatter(X_circles[:, 0], X_circles[:, 1], c=labels_sklearn, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[1].set_title('Scikit-learn', fontweight='bold')
plt.tight_layout()
plt.show()

print("From-scratch Spectral Clustering validated!")


---
# Part 11: Cluster Quality Evaluation

How do we measure clustering quality?

## 11.1 Silhouette Score (No Labels Required)

Measures how similar a point is to its own cluster compared to other clusters.

**Formula:** s(i) = (b(i) - a(i)) / max{a(i), b(i)}
- a(i): Mean distance to points in same cluster
- b(i): Mean distance to points in nearest cluster
- Range: [-1, +1], higher is better
- +1: Point is well-clustered
- 0: Point is on cluster boundary
- -1: Point is in wrong cluster

In [ ]:
# Evaluate clustering quality on Iris
from sklearn.datasets import load_iris

iris = load_iris()
X_iris = iris.data
y_true = iris.target

# Try different k values
k_values = range(2, 7)
silhouette_scores = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_iris)
    score = silhouette_score(X_iris, labels)
    silhouette_scores.append(score)

# Plot
plt.figure(figsize=(10, 5))
plt.plot(k_values, silhouette_scores, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Number of Clusters (k)', fontsize=12)
plt.ylabel('Silhouette Score', fontsize=12)
plt.title('Silhouette Score vs Number of Clusters', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

best_k = k_values[np.argmax(silhouette_scores)]
print(f"Best k based on Silhouette Score: {best_k}")
print(f"Silhouette scores: {dict(zip(k_values, [f'{s:.3f}' for s in silhouette_scores]))}")

## 11.2 Purity (Requires Ground Truth)

**Purity** measures how homogeneous clusters are.

**Formula:** Purity = (1/N) Σ max_j |C_i ∩ L_j|

- Range: [0, 1], higher is better
- Perfect clustering: purity = 1
- Problem: Trivial solution (each point its own cluster) gives purity = 1

In [ ]:
def purity_score(y_true, y_pred):
    # Compute confusion matrix
    contingency_matrix = np.zeros((len(np.unique(y_true)), len(np.unique(y_pred))))
    for i, true_label in enumerate(np.unique(y_true)):
        for j, pred_label in enumerate(np.unique(y_pred)):
            contingency_matrix[i, j] = np.sum((y_true == true_label) & (y_pred == pred_label))
    
    # Purity is sum of max values in each column divided by total
    return np.sum(np.max(contingency_matrix, axis=0)) / np.sum(contingency_matrix)

# Test on Iris
kmeans_iris = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_iris = kmeans_iris.fit_predict(X_iris)

purity = purity_score(y_true, labels_iris)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# True labels
axes[0].scatter(X_iris[:, 0], X_iris[:, 1], c=y_true, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[0].set_title('True Labels', fontweight='bold')
axes[0].set_xlabel('Sepal Length')
axes[0].set_ylabel('Sepal Width')

# Predicted clusters
axes[1].scatter(X_iris[:, 0], X_iris[:, 1], c=labels_iris, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[1].set_title(f'K-Means Clusters (Purity={purity:.3f})', fontweight='bold')
axes[1].set_xlabel('Sepal Length')
axes[1].set_ylabel('Sepal Width')

plt.tight_layout()
plt.show()

print(f"Purity Score: {purity:.3f}")
print("Higher purity means clusters are more homogeneous")


## 11.3 Entropy (Requires Ground Truth)

**Entropy** measures the disorder within clusters.

**Formula:** Entropy = Σ (N_i/N) × e(C_i)

where e(C_i) = -Σ p_ij log(p_ij)

- Range: [0, log(k)], lower is better
- Perfect clustering: entropy = 0

In [ ]:
def entropy_score(y_true, y_pred):
    from scipy.stats import entropy as scipy_entropy
    
    total_entropy = 0
    n_samples = len(y_true)
    
    for cluster_label in np.unique(y_pred):
        cluster_mask = y_pred == cluster_label
        cluster_size = np.sum(cluster_mask)
        
        if cluster_size == 0:
            continue
        
        # Distribution of true labels in this cluster
        label_counts = np.bincount(y_true[cluster_mask], minlength=len(np.unique(y_true)))
        label_probs = label_counts / cluster_size
        
        # Entropy of this cluster
        cluster_entropy = scipy_entropy(label_probs, base=2)
        
        # Weighted by cluster size
        total_entropy += (cluster_size / n_samples) * cluster_entropy
    
    return total_entropy

entropy = entropy_score(y_true, labels_iris)
print(f"Entropy Score: {entropy:.3f}")
print(f"Purity Score: {purity:.3f}")
print("\nLower entropy = better clustering")
print("Higher purity = better clustering")


## 11.4 RAND Index (Requires Ground Truth)

**RAND Index** measures agreement between two clusterings.

**Formula:** RAND = (A + B) / (A + B + C + D)

where:
- A: Pairs in same cluster in both
- B: Pairs in different clusters in both
- C: Pairs in same cluster in predicted, different in true
- D: Pairs in different clusters in predicted, same in true

**Adjusted RAND Index** corrects for chance.

In [ ]:
from sklearn.metrics import rand_score, adjusted_rand_score

rand = rand_score(y_true, labels_iris)
adj_rand = adjusted_rand_score(y_true, labels_iris)

print(f"RAND Index: {rand:.3f}")
print(f"Adjusted RAND Index: {adj_rand:.3f}")
print("\nAdjusted RAND Index:")
print("• Range: [-1, 1]")
print("• 1: Perfect match")
print("• 0: Random labeling")
print("• Negative: Worse than random")


## 11.5 Silhouette Score (No Ground Truth Required)

We already covered this, but let's add silhouette plot visualization.

In [ ]:
from sklearn.metrics import silhouette_samples

# Compute silhouette for each sample
silhouette_vals = silhouette_samples(X_iris, labels_iris)
silhouette_avg = np.mean(silhouette_vals)

# Create silhouette plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Silhouette plot
y_lower = 10
for i in range(3):
    cluster_silhouette_vals = silhouette_vals[labels_iris == i]
    cluster_silhouette_vals.sort()
    
    size_cluster_i = cluster_silhouette_vals.shape[0]
    y_upper = y_lower + size_cluster_i
    
    color = plt.cm.viridis(float(i) / 3)
    axes[0].fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_silhouette_vals,
                          facecolor=color, edgecolor=color, alpha=0.7)
    
    axes[0].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10

axes[0].set_title('Silhouette Plot', fontweight='bold')
axes[0].set_xlabel('Silhouette Coefficient')
axes[0].set_ylabel('Cluster')
axes[0].axvline(x=silhouette_avg, color='red', linestyle='--', label=f'Average: {silhouette_avg:.3f}')
axes[0].legend()

# Scatter plot colored by silhouette score
scatter = axes[1].scatter(X_iris[:, 0], X_iris[:, 1], c=silhouette_vals, cmap='RdYlGn', 
                         s=50, alpha=0.6, edgecolors='k', vmin=-0.2, vmax=1)
axes[1].set_title('Points Colored by Silhouette Score', fontweight='bold')
axes[1].set_xlabel('Sepal Length')
axes[1].set_ylabel('Sepal Width')
plt.colorbar(scatter, ax=axes[1])

plt.tight_layout()
plt.show()

print(f"Average Silhouette Score: {silhouette_avg:.3f}")
print("Green points: Well-clustered")
print("Yellow/Red points: Poorly clustered or misassigned")


## 11.6 Comprehensive Comparison

Let's compare all metrics on Iris dataset.

In [ ]:
# Compute all metrics
from sklearn.metrics import adjusted_mutual_info_score, fowlkes_mallows_score

metrics = {
    'Purity': purity,
    'Entropy': entropy,
    'RAND Index': rand,
    'Adjusted RAND': adj_rand,
    'Silhouette': silhouette_avg,
    'Adj. Mutual Info': adjusted_mutual_info_score(y_true, labels_iris),
    'Fowlkes-Mallows': fowlkes_mallows_score(y_true, labels_iris)
}

# Display as table
print("\nCluster Quality Metrics for K-Means on Iris:")
print("=" * 50)
for metric, value in metrics.items():
    print(f"{metric:20s}: {value:6.3f}")
print("=" * 50)

print("\nInterpretation:")
print("• Purity, RAND, Silhouette, AMI, FM: Higher is better")
print("• Entropy: Lower is better")
print("• Adjusted metrics correct for chance")


## 11.7 Summary: Choosing the Right Metric

| Metric | Requires Labels | Range | Best Value | Use When |
|--------|----------------|-------|------------|----------|
| **Purity** | Yes | [0, 1] | 1 | Simple interpretation |
| **Entropy** | Yes | [0, log k] | 0 | Information-theoretic view |
| **RAND Index** | Yes | [0, 1] | 1 | Pair-wise agreement |
| **Adj. RAND** | Yes | [-1, 1] | 1 | Corrects for chance |
| **Silhouette** | No | [-1, 1] | 1 | No ground truth available |
| **Adj. MI** | Yes | [0, 1] | 1 | Information-based, corrected |

**Recommendations:**
- **With ground truth**: Use Adjusted RAND or Adjusted Mutual Information
- **Without ground truth**: Use Silhouette Score
- **For interpretation**: Use Purity (easy to understand)
- **For research**: Report multiple metrics

---
# Part 12: Algorithm Comparison Summary

| Algorithm | Cluster Shape | Requires k | Handles Noise | Time Complexity | Best For |
|-----------|---------------|------------|---------------|-----------------|----------|
| **K-Means** | Spherical | Yes | No | O(nkt) | Large datasets, spherical clusters |
| **Hierarchical** | Any | No | No | O(n²log n) | Small datasets, hierarchy needed |
| **DBSCAN** | Arbitrary | No | Yes | O(n log n) | Arbitrary shapes, noise detection |
| **Spectral** | Non-convex | Yes | No | O(n³) | Graph data, complex shapes |

## Decision Guide

**Use K-Means when:**
- Clusters are roughly spherical
- You know k in advance
- You have large datasets
- Speed is important

**Use Hierarchical when:**
- You need a hierarchy of clusters
- You don't know k
- Dataset is small (< 10,000 points)
- You want to explore different k values

**Use DBSCAN when:**
- Clusters have arbitrary shapes
- You need to identify outliers
- Cluster density is roughly uniform
- You don't know k

**Use Spectral when:**
- Data has graph structure
- Clusters are non-convex
- You can afford computational cost
- K-Means fails on your data

---
## Summary

**Key Takeaways:**

1. **No single best algorithm** - choice depends on data characteristics
2. **K-Means** is fast and simple but assumes spherical clusters
3. **Hierarchical** provides flexibility but is computationally expensive
4. **DBSCAN** handles arbitrary shapes and noise but sensitive to parameters
5. **Spectral** works on complex shapes but requires eigendecomposition
6. **Always visualize** your data and results
7. **Use multiple metrics** to evaluate clustering quality

**Further Reading:**
- Scikit-learn clustering documentation
- "Pattern Recognition and Machine Learning" by Bishop
- "The Elements of Statistical Learning" by Hastie et al.